# Lab. 3-1 Schema Preparation-1

This lab focuses on steps 1 and 2 of the Text2SQL process, specifically dealing with Schema Preparation.

Schema Linking, which involves organizing the necessary schema for query generation, is highlighted as one of the most challenging aspects of Text2SQL for complex databases. It addresses real-world challenges in corporate environments, such as:
1. Abbreviated or unclear table/column names
2. Too many tables/columns to include in a single prompt

To address this, we need to refine schema description documents tailored to our database and select the necessary context to provide to the LLM. In this notebook, we will simulate the schema preparation process using the Chinook DB description document. This is part of a larger workflow that will continue in subsequent labs

![Intro](../images/text2sql/schema-prep-1.png)


## Step 0: OpenSearch Configurations

In [1]:
!pip install -q opensearch-py

In [ ]:
from libs.ssm import parameter_store

pm = parameter_store('us-west-2')
domain_endpoint = pm.get_params(key="chatbot-opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="chatbot-opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="chatbot-opensearch_user_password", enc=True)
print(opensearch_domain_endpoint)

## Step 1: Load Schema Description Document (Corresponding to 1. Schema Loader in the above diagram)

Companies may have schema description documents defined in formats like Excel or CSV. Let's assume we parse these and convert them into the following Schema Description format:

```
{
    "table_name": {
        "table_desc": "Description of the table",
        "cols": [
            {
                "col": "Column Name 1",
                "col_desc": "Description of the column including PK info"
            },
            {
                "col": "Column Name 2",
                "col_desc": "Description of the column"
            }
        ]
    }
}
```

The initial description document should include the table names, basic descriptions of the tables, column names, and descriptions of the columns. If a company doesn't have a well-organized schema description document, we could provide very basic information and have an LLM augment it to generate the initial description document itself. 

For the LLM call script to do this, refer to this [link](https://github.com/kevmyung/db-schema-loader).

In [ ]:
import json

file_path = './chinook_schema.json'

with open(file_path, 'r') as file:
    schema_description = json.load(file)

print(json.dumps(schema_description, indent=4))

#### Now we will continue with the follow-up tasks using the Schema Description document


## Step 2: Translating Sample SQL Queries to Natural Language Questions 

Providing good sample queries to the LLM is helpful not only for query writing but also for Schema Linking.

However, since Text2SQL hasn't been used before, there are no natural language questions matched to existing SQL queries. Only the frequently used SQL queries are available.

In Step 2, we will proceed with the SQL2Text process, converting these frequently used queries into natural language questions. (Corresponding to 3. Query Translator in the above diagram)

In [ ]:
sql_file = './chinook_sample_queries.sql'

with open(sql_file, 'r') as file:
    data = file.read()

queries = [query.strip() for query in data.split(';') if query.strip()]

for i, query in enumerate(queries, start=1):
    print(f"Query {i}:\n{query}\n{'-'*80}\n")

To interpret the queries, we extract the table/column information used in each query as follows:
```
{
  "table": ["table1", "table2", ...],
  "column": ["col1", "col2", ...]
}
```
The following is the LLM request syntax for extracting the schema list used in SQL queries.

In [ ]:
extraction_sys_prompt = [{
    "text": """ 
You are an expert in extracting table names and column names from SQL queries. 
From the provided SQL query, extract all table names and column names used for SELECT, WHERE, and JOIN clauses, excluding asterisks ("*"). 
Ensure that the response is in a valid JSON format that can be used directly with json.load(). 
Skip the preamble and only provide the answer in a JSON document:

{
  "table": ["table1", "table2", ...],
  "column": ["col1", "col2", ...]
}

<input>
SQL:
SELECT * from sample_table 
where sample_column like '%something%'
LIMIT 200;
</input>

<output>
{
  "table": ["sample_table"],
  "column": ["sample_column"]
}
</output>
""" 
}]


def get_extraction_prompt(sql):
    return [{
        "role": "user",
        "content": [{"text": f"SQL: \n{sql}"}]
    }]

In [ ]:
import boto3
from botocore.config import Config

region_name = "us-west-2"
llm_model = "anthropic.claude-3-5-haiku-20241022-v1:0"

def init_boto3_client(region: str):
    retry_config = Config(
        region_name=region,
        retries={"max_attempts": 10, "mode": "standard"}
    )
    return boto3.client("bedrock-runtime", region_name=region, config=retry_config)

def converse_with_bedrock(boto3_client, sys_prompt, usr_prompt):    
    temperature = 0.0
    top_p = 0.1
    inference_config = {"temperature": temperature, "topP": top_p}
    
    response = boto3_client.converse(
        modelId=llm_model, 
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config
    )

    return response['output']['message']['content'][0]['text']

In [ ]:
boto3_client = init_boto3_client(region_name)

sql = queries[8].strip()

response = converse_with_bedrock(boto3_client, extraction_sys_prompt, get_extraction_prompt(sql))
used_schema = json.loads(response)
print(used_schema)

For example, let's extract the schema used in the query below.

```SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5``` 

#### Now, let's look at the description of the schema used in this query

In [ ]:
def extract_descriptions(table_info, tables, columns):
    tables_lower = {table.lower() for table in tables}
    columns_lower = {column.lower() for column in columns}
    
    description = {
        "table": {},
        "column": {}
    }
    
    for table_schema in table_info:
        for table_name, table_info in table_schema.items():
            if table_name.lower() in tables_lower:
                description["table"][table_name] = table_info["table_desc"]
                for col in table_info["cols"]:
                    col_name = col["col"]
                    if col_name.lower() in columns_lower:
                        description["column"][col_name] = col["col_desc"]
    return description

In [ ]:
extracted_description = extract_descriptions(schema_description, used_schema['table'], used_schema['column'])
print(extracted_description)

#### The next step is to ask for a natural language interpretation of the query

In [2]:
translation_sys_prompt = [{
    "text": """  
You are an SQL expert who can understand the intent behind a given SQL query. 
Translate the SQL query into a natural language request that a real user might make. 

- Keep your translation concise and conversational, mimicking how an actual user would ask for the information sought by the query. 
- Do not reference the <description> section directly and do not use a question form. 
- Ensure to include all conditions specified in the SQL query in the request.
- Write possible business and functional purposes of the query.
- Write very detailed purposes and motives of the query in detail.
- Skip the preamble and phrase only the natural language request using a concise and straightforward tone without a verb ending. 
"""
}]


def get_translation_prompt(description, sql):
    return [{
        "role": "user",
        "content": [{"text": f"<description>{description}<description>\n\n SQL: {sql}"}]
    }]

In [ ]:
region_name = "us-west-2"
llm_model = "anthropic.claude-3-5-haiku-20241022-v1:0"

response = converse_with_bedrock(boto3_client, translation_sys_prompt, get_translation_prompt(extracted_description, queries[8]))

print(response)

#### The natural language description for the following query was defined by the LLM as shown above.

`SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5`

#### Below is a script that repeats the above process for all SQL queries. (Takes about 1-2 minutes)

In [ ]:
import os

FILE_PATH_1 = './example_queries_temp.jsonl'
def query_translation(table_info, queries):
    if os.path.exists(FILE_PATH_1):
        os.remove(FILE_PATH_1)

    with open(FILE_PATH_1, 'a') as output_file:
        for query in queries:
            sql = query.strip()
            
            try:
                response = converse_with_bedrock(boto3_client, extraction_sys_prompt, get_extraction_prompt(sql))
                schema = json.loads(response)
            except json.JSONDecodeError:
                print(response)

            description = extract_descriptions(table_info, schema["table"], schema["column"])

            input = converse_with_bedrock(boto3_client, translation_sys_prompt, get_translation_prompt(description, sql))
            
            # Write input and query to the file in JSON format
            data = {"input": input, "query": sql}
            output_file.write(json.dumps(data, ensure_ascii=False) + "\n")
            
query_translation(schema_description, queries)

The results of the completed query transformations are stored in the `./lab3_text2sql_schema_preparation/example_queries_temp.jsonl` file.

In [ ]:
with open(FILE_PATH_1, 'r') as file:
    for line in file:
        data = json.loads(line)
        print(data)

## Step 3: Sample Query Vector Embedding and OpenSearch Storage

We need to embed the natural language questions from the <natural language question & SQL query> combinations into vectors. 

This is to facilitate finding SQL queries similar to user questions. The following code initializes the OpenSearch environment. (Creating connections and initializing Index)

In [ ]:
import yaml
from opensearchpy import OpenSearch, RequestsHttpConnection

INDEX_NAME = "example_queries"

def load_opensearch_config():
    with open("./libs/opensearch.yml", 'r', encoding='utf-8') as file:
        return yaml.safe_load(file)

def init_opensearch(config):
    mapping = {"settings": config['settings'], "mappings": config['mappings-sql']}
    endpoint = opensearch_domain_endpoint
    http_auth = (opensearch_user_id, opensearch_user_password)

    os_client = OpenSearch(
            hosts=[{'host': endpoint.replace("https://", ""),'port': 443}],
            http_auth=http_auth, 
            use_ssl=True,
            verify_certs=True,
            timeout=300,
            connection_class=RequestsHttpConnection
    )

    create_os_index(os_client, mapping)
    return os_client

def create_os_index(os_client, mapping):
    exists = os_client.indices.exists(INDEX_NAME)

    if exists:
        os_client.indices.delete(index=INDEX_NAME)
        print("Existing index has been deleted. Create new one.")
    else:
        print("Index does not exist, Create one.")

    os_client.indices.create(INDEX_NAME, body=mapping)

config = load_opensearch_config()
os_client = init_opensearch(config)

We will convert the previously created <natural language question & SQL query> pairs into vector embeddings, and format them into a Data-Action format suitable for bulk indexing in OpenSearch.

In [ ]:
FILE_PATH_2 = './example_queries.jsonl'

embed_model = "amazon.titan-embed-text-v2:0"
region_name = "us-west-2"

def input_embedding():
    num = 0
    if os.path.exists(FILE_PATH_2):
        os.remove(FILE_PATH_2)

    with open(FILE_PATH_1, 'r') as input_file, open(FILE_PATH_2, 'a') as output_file:
        for line in input_file:
            
            data = json.loads(line)
            input = data['input']
            query = data['query']

            response = boto3_client.invoke_model(
                modelId=embed_model,
                body=json.dumps({"inputText": input})
            )

            # Data part
            body = { "input": input, "query": query, "input_v": json.loads(response['body'].read())['embedding'] }

            # Action part
            action = { "index": { "_index": INDEX_NAME, "_id": str(num) } }

            # Write action and body to the file in correct bulk format
            output_file.write(json.dumps(action, ensure_ascii=False) + "\n")
            output_file.write(json.dumps(body, ensure_ascii=False) + "\n")

            num += 1    

input_embedding()

In the `./lab3_text2sql_schema_preparation/example_queries.jsonl` file, you can see the converted embeddings.

In [ ]:
with open(FILE_PATH_2, 'r') as file:
    bulk_data = file.read()
        
response = os_client.bulk(body=bulk_data)
if response["errors"]:
    print("There were errors during bulk indexing:")
    for item in response["items"]:
        if 'index' in item and item['index']['status'] >= 400:
            print(f"Error: {item['index']['error']['reason']}")
else:
    print("Bulk-inserted all items successfully.")

#### Now, the sample queries have been stored into OpenSearch